In [19]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get

import itertools
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 600)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, svm, tree
from sklearn.ensemble import RandomForestRegressor
from  sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
sns.set(rc={'figure.figsize':(16,9)})
#import swifter

np.random.seed(1907)

In [20]:
def initial_data_preparation():
    

    data = pd.read_csv("../data/interim/spark_processed_data.csv", sep="|")

    
    
    print(f"{len(data)} Rows.")
   

    multi_choice_columns = pd.read_csv("../data/interim/spark_multi_choice_columns.csv", sep="|")["0"].tolist()
    
    data = data.drop(multi_choice_columns, axis=1)
    
    # Preserve country columns for target transformation
    data["Country_for_target_transformation"] = data["Country"]
    
    #Drop observations have less then $100 ConvertedComp and professional dev.
    data = data[(~(data["ConvertedComp"]<100) &  (data.MainBranch == "I am a developer by profession"))]

    
    
    # One hot encoding
    cat_columns = ["Country", "Gender", "JobSat", 
                                        "JobSeek",
                                        "Employment",
                                        "MainBranch", 
                                        "Hobbyist", 
                                        "EdLevel", 
                                        "NEWDevOps", 
                                        "NEWDevOpsImpt", 
                                        "NEWEdImpt", 
                                        "NEWLearn",
                                        "NEWOffTopic",
                                        "NEWOnboardGood",
                                        "NEWOtherComms",
                                        "NEWOvertime",
                                        "NEWPurpleLink","OpSys","OrgSize",
                                        "PurchaseWhat","SOAccount",
                                        "SOComm","SOPartFreq","SOVisitFreq",
                                        "SurveyEase", "SurveyLength","Trans", "UndergradMajor","WelcomeChange"]
    
    cat_columns = [col for col in cat_columns if col not in multi_choice_columns]
    data = pd.get_dummies(data=data, columns=cat_columns)

    
    data = data.dropna(subset=["ConvertedComp"], axis=0)
    data = data[data["ConvertedComp"] >0]
    
    #Drop countries that has less than 150 observation
    countries = data["Country_for_target_transformation"].value_counts()
    countries_to_drop = countries[countries <= 200].index.tolist()
    data = data[~data["Country_for_target_transformation"].isin(countries_to_drop)].reset_index(drop=True)
    
    
    print(f"{len(data)} Rows.")
    
    
    # Fill NaNs with median
    nas = data.isna().sum()
    nas = nas[nas > 0]

    for na_col in nas.index:
        data[na_col].fillna(data[na_col].median(), inplace=True)

    #LABEL :"ConvertedComp"
    return data

In [21]:
def load_processed_data(force_processing=False):
    if force_processing:
        data = initial_data_preparation()
        data.to_csv("../data/processed/processed_data.csv", sep="|", index=False)
    else:
        try:
            data = pd.read_csv("../data/processed/processed_data.csv", sep="|")
        except Exception as e:
            print(e)
            data = initial_data_preparation()
            data.to_csv("../data/processed/processed_data.csv", sep="|", index=False)
    
    return data

data = load_processed_data(force_processing=False)

In [22]:
data.head()

,Age,Age1stCode,ConvertedComp,WorkWeekHrs,YearsCode,YearsCodePro,DatabaseDesireNextYear_MariaDB,DatabaseDesireNextYear_Microsoft SQL Server,DatabaseDesireNextYear_MongoDB,DatabaseDesireNextYear_MySQL,DatabaseDesireNextYear_Oracle,DatabaseDesireNextYear_PostgreSQL,DatabaseDesireNextYear_Redis,DatabaseDesireNextYear_SQLite,DatabaseDesireNextYear_Cassandra,DatabaseDesireNextYear_IBM DB2,DatabaseDesireNextYear_DynamoDB,DatabaseDesireNextYear_Elasticsearch,DatabaseDesireNextYear_Firebase,DatabaseDesireNextYear_Couchbase,DatabaseWorkedWith_Couchbase,DatabaseWorkedWith_Microsoft SQL Server,DatabaseWorkedWith_MySQL,DatabaseWorkedWith_PostgreSQL,DatabaseWorkedWith_DynamoDB,DatabaseWorkedWith_Elasticsearch,DatabaseWorkedWith_Firebase,DatabaseWorkedWith_MariaDB,DatabaseWorkedWith_SQLite,DatabaseWorkedWith_MongoDB,DatabaseWorkedWith_Oracle,DatabaseWorkedWith_Redis,DatabaseWorkedWith_IBM DB2,DatabaseWorkedWith_Cassandra,DevType_Scientist,"DevType_Developer, back-end","DevType_Developer, desktop or enterprise applications","DevType_Developer, front-end","DevType_Developer, full-stack","DevType_Developer, game or graphics","DevType_Developer, mobile",DevType_Academic researcher,DevType_Database administrator,DevType_Designer,"DevType_Developer, QA or test","DevType_Engineer, data",DevType_Data or business analyst,DevType_Product manager,DevType_Data scientist or machine learning specialist,"DevType_Developer, embedded applications or devices",DevType_DevOps specialist,DevType_System administrator,DevType_Senior executive/VP,"DevType_Engineer, site reliability",DevType_Engineering manager,DevType_Educator,DevType_Marketing or sales professional,Ethnicity_White or of European descent,Ethnicity_South Asian,Ethnicity_East Asian,Ethnicity_Biracial,Ethnicity_Multiracial,Ethnicity_Black or of African descent,Ethnicity_Southeast Asian,Ethnicity_Middle Eastern,"Ethnicity_Indigenous (such as Native American, Pacific Islander, or Indigenous Australian)",Ethnicity_Hispanic or Latino/a/x,Gender_Man,"Gender_Non-binary, genderqueer, or gender non-conforming",Gender_Woman,JobFactors_Industry that I’d be working in,"JobFactors_Languages, frameworks, and other technologies I’d be working with",JobFactors_How widely used or impactful my work output would be,JobFactors_Office environment or company culture,JobFactors_Opportunities for professional development,JobFactors_Remote work options,JobFactors_Diversity of the company or organization,JobFactors_Specific department or team I’d be working on,JobFactors_Financial performance or funding status of the company or organization,JobFactors_Family friendliness,JobFactors_Flex time or a flexible schedule,LanguageDesireNextYear_C#,LanguageDesireNextYear_Bash/Shell/PowerShell,LanguageDesireNextYear_JavaScript,LanguageDesireNextYear_Python,LanguageDesireNextYear_R,LanguageDesireNextYear_SQL,LanguageDesireNextYear_C,LanguageDesireNextYear_C++,LanguageDesireNextYear_Dart,LanguageDesireNextYear_HTML/CSS,LanguageDesireNextYear_Java,LanguageDesireNextYear_Kotlin,LanguageDesireNextYear_Objective-C,LanguageDesireNextYear_Perl,LanguageDesireNextYear_Swift,LanguageDesireNextYear_TypeScript,LanguageDesireNextYear_PHP,LanguageDesireNextYear_Haskell,LanguageDesireNextYear_Go,LanguageDesireNextYear_Rust,LanguageDesireNextYear_Julia,LanguageDesireNextYear_VBA,LanguageDesireNextYear_Ruby,LanguageDesireNextYear_Scala,LanguageDesireNextYear_Assembly,LanguageWorkedWith_Assembly,LanguageWorkedWith_Bash/Shell/PowerShell,LanguageWorkedWith_C,LanguageWorkedWith_C#,LanguageWorkedWith_C++,LanguageWorkedWith_HTML/CSS,LanguageWorkedWith_JavaScript,LanguageWorkedWith_PHP,LanguageWorkedWith_Python,LanguageWorkedWith_SQL,LanguageWorkedWith_TypeScript,LanguageWorkedWith_VBA,LanguageWorkedWith_R,LanguageWorkedWith_Java,LanguageWorkedWith_Kotlin,LanguageWorkedWith_Scala,LanguageWorkedWith_Swift,LanguageWorkedWith_Ruby,LanguageWorkedWith_Objective-C,LanguageWorkedWith_Dart,LanguageWorkedWith_Go,LanguageWorkedWith_Perl,LanguageWorkedWith_Rust,Language

## TODO

- Dev Type'a göre en çok kullanılan teknolojiler ve herkes bunlardan ne kadarını biliyor. **Erdem**
- Sunum Hazırlama **Bengi**
- Rapor **Berk**

In [23]:
def target_transformation(x_train, x_test, y_train, y_test):
    train_set = pd.concat([x_train, y_train], axis=1)
    test_set = pd.concat([x_test, y_test], axis=1)
    
    avg_prf_comp_by_country = train_set.groupby("Country_for_target_transformation").ConvertedComp.median()
    target_transformer = lambda row: (row['ConvertedComp'] / avg_prf_comp_by_country[row["Country_for_target_transformation"]]) *100
    
    y_train = train_set.apply(target_transformer, axis=1 ).rename("ConvertedComp_index")
    y_test = test_set.apply(target_transformer, axis=1 ).rename("ConvertedComp_index")
     
    
    return x_train, x_test, y_train, y_test
    

In [24]:
def calculate_high_and_low_leverage_tech(data):
            technologies = [feat for feat in data.columns if "WorkedWith" in feat]
            us_data = data[data["Country_for_target_transformation"] =="United States"]
            tech_leverages = {}
            for tech in technologies:
                tech_median = us_data.groupby(tech).ConvertedComp.median()
                if len(tech_median) != 2: continue
                tech_leverage = (tech_median.loc[1] / tech_median.loc[0]) -1
                tech_leverages[tech] = tech_leverage
            tech_leverages = pd.Series(tech_leverages).sort_values(ascending=False)

            high_leverage_technologies = tech_leverages[tech_leverages >= tech_leverages.quantile(0.80)].index.tolist()
            low_leverage_technologies = tech_leverages[tech_leverages <= tech_leverages.quantile(0.20)].index.tolist()
            return high_leverage_technologies, low_leverage_technologies

In [25]:
def prepare_data(data=data, 
                 exclude_columns=None, 
                 features_to_use=None, 
                 apply_x_col=None, 
                 columns_to_bin=None,
                 transform_target=False,
                 feat_agg_calculate_chosen_options_ratio=False,
                 feat_agg_worked_with_high_and_low_leverage=False):

    
    if exclude_columns is not None and features_to_use is not None:
        raise Exception("exclude_columns and features_to_use cannot be used at the same time.")

    if exclude_columns is not None:
        data = data.drop(exclude_columns, axis=1)

    if features_to_use is not None:
        data = data[features_to_use + ["revenue"]]
        
    if feat_agg_calculate_chosen_options_ratio:
        def proportion_chosen_options(columns_contains):
            desire_next_year_feats = [feat for feat in data.columns if columns_contains in feat]
            return data[desire_next_year_feats].sum(axis=1) / len(desire_next_year_feats)
        
        data["feat_agg_proportion_chosen_DesireNextYear"] = proportion_chosen_options("DesireNextYear")
        data["feat_agg_proportion_chosen_LanguageDesireNextYear"] = proportion_chosen_options("LanguageDesireNextYear")
        data["feat_agg_proportion_chosen_PlatformDesireNextYear"] = proportion_chosen_options("PlatformDesireNextYear")
        data["feat_agg_proportion_chosen_WebframeDesireNextYear"] = proportion_chosen_options("WebframeDesireNextYear")
        data["feat_agg_proportion_chosen_DatabaseDesireNextYear"] = proportion_chosen_options("DatabaseDesireNextYear")

        data["feat_agg_proportion_chosen_WorkedWith"] = proportion_chosen_options("WorkedWith")
        data["feat_agg_proportion_chosen_LanguageWorkedWith"] = proportion_chosen_options("LanguageWorkedWith")
        data["feat_agg_proportion_chosen_PlatformWorkedWith"] = proportion_chosen_options("PlatformWorkedWith")
        data["feat_agg_proportion_chosen_WebframeWorkedWith"] = proportion_chosen_options("WebframeWorkedWith")
        data["feat_agg_proportion_chosen_DatabaseWorkedWith"] = proportion_chosen_options("DatabaseWorkedWith")
        
        

    x = data.drop('ConvertedComp', axis=1)
    
    y = data['ConvertedComp']

    if apply_x_col is not None:
        x = x.apply(apply_x_col)
    

    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size = 0.33, 
                                                        random_state = 3, 
                                                        stratify=x["Country_for_target_transformation"])
    
    if feat_agg_worked_with_high_and_low_leverage:
        
        high_leverage_technologies, low_leverage_technologies = calculate_high_and_low_leverage_tech(pd.concat([x_train, y_train], 
                                                                                                               axis=1))
        def proportion_chosen_given_Tech(technologies):
            return data[technologies].sum(axis=1) / len(technologies)
        
        x_train["feat_agg_high_leverage_tech_worked"] = proportion_chosen_given_Tech(high_leverage_technologies)
        x_train["feat_agg_low_leverage_tech_worked"] = proportion_chosen_given_Tech(low_leverage_technologies)
        
        
        x_test["feat_agg_high_leverage_tech_worked"] = proportion_chosen_given_Tech(high_leverage_technologies)
        x_test["feat_agg_low_leverage_tech_worked"] = proportion_chosen_given_Tech(low_leverage_technologies)
        
        technologies = [feat for feat in data.columns if (("WorkedWith" in feat) or ("DesireNextYear" in feat))]
        x_train = x_train.drop(technologies, axis=1)
        x_test = x_test.drop(technologies, axis=1)
    
    
    if transform_target:
        x_train, x_test, y_train, y_test = target_transformation(x_train, x_test, y_train, y_test)
    
    #Preserved from initial data preparation for target_transformation
    x_train.drop(["Country_for_target_transformation"], axis=1, inplace=True)
    x_test.drop(["Country_for_target_transformation"], axis=1, inplace=True)
        
    

    if columns_to_bin is not None:
        for column in columns_to_bin:
            if column not in x_train.columns: continue
            x_train.loc[:,column], bins_ = pd.qcut(x_train.loc[:,column], q=4,  retbins=True, duplicates="drop")
            x_test.loc[:,column] = pd.cut(x_test.loc[:,column], bins=bins_, )
            
            x_train = pd.get_dummies(x_train)
            x_test = pd.get_dummies(x_test)
    
    return x_train, x_test, y_train, y_test, data

In [26]:
def scale_data(x_train, x_test, scaler=StandardScaler):

    scaler = scaler()
    scaler.fit(x_train)

    x_train_scaled = scaler.transform(x_train)
    x_train_scaled = pd.DataFrame(x_train_scaled, index=x_train.index, columns=x_train.columns)

    x_test_scaled = scaler.transform(x_test)
    x_test_scaled = pd.DataFrame(x_test_scaled, index=x_test.index, columns=x_test.columns)

    x_train_scaled.fillna(x_train_scaled.median(), inplace=True)
    x_test_scaled.fillna(x_test_scaled.median(), inplace=True)

    return x_train_scaled, x_test_scaled

In [27]:
def evaluate_regression(y_true_train, y_pred_train, y_true_test, y_pred_test, model_name="", model=None, num_feat=""):
    mape_train = np.abs((y_true_train - y_pred_train) / y_true_train).mean(axis=0)
    mape_test = np.abs((y_true_test - y_pred_test) / y_true_test).mean(axis=0)
    return pd.DataFrame.from_records([[#metrics.mean_squared_error(y_true_train, y_pred_train),
                                        mape_train,
                                       metrics.mean_absolute_error(y_true_train, y_pred_train),
                                       metrics.max_error(y_true_train, y_pred_train),
                                       metrics.r2_score(y_true_train, y_pred_train), 
                                      #metrics.mean_squared_error(y_true_test, y_pred_test),
                                        mape_test,
                                       metrics.mean_absolute_error(y_true_test, y_pred_test),
                                       metrics.max_error(y_true_test, y_pred_test),
                                       metrics.r2_score(y_true_test, y_pred_test),
                                       model,
                                       num_feat]],
                                     
                                     index=[model_name], 
                                     columns=[#'mean_squared_error_train', 
                                             "mape_train",
                                              'mean_absolute_error_train', 
                                              'max_error_train', 
                                              "r2_score_train",
                                              #'mean_squared_error_test', 
                                                 "mape_test",
                                              'mean_absolute_error_test', 
                                              'max_error_test', 
                                              "r2_score_test", 
                                              "model_object", 
                                              "num_feat"])


In [28]:
def run_models(x_train, x_test, y_train, y_test, feture_elimination_num_feat=None):
        results = pd.DataFrame()
        for model in [#linear_model.LinearRegression(), 
                        linear_model.RidgeCV(),
                        linear_model.LassoCV(),
                        svm.LinearSVR(),
                        #svm.SVR(kernel="rbf",),
                        #svm.SVR(kernel="poly"),
                        tree.DecisionTreeRegressor(),
                        RandomForestRegressor()
                     ]:
                
                if feture_elimination_num_feat is not None :
                        try:
                                if feture_elimination_num_feat == "auto":
                                        feture_elimination_num_feat = np.linspace(5,len(x_train.columns),5, dtype=int)

                                for n_features_to_select in feture_elimination_num_feat:
                                        model_ = RFE(estimator=model, n_features_to_select=n_features_to_select)
                                        model_.fit(x_train, y_train)
                                        y_train_pred = model_.predict(x_train)
                                        y_test_pred = model_.predict(x_test)

                                        results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                                                                model_name=type(model_).__name__, model=model_, num_feat=n_features_to_select))
                                return results
                        except Exception as e:
                            print(e)
                                
                        
               
                model.fit(x_train, y_train)
                
                y_train_pred = model.predict(x_train)
                y_test_pred = model.predict(x_test)

                results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                                        model_name=type(model).__name__, model=model, num_feat=len(x_train.columns) ))
        return results


In [29]:
experiment_list = pd.DataFrame()

def run(exclude_columns=None, 
        features_to_use=None, 
        name="", 
        apply_x_col=None, 
        x_train_=None, 
        y_train_=None, 
        columns_to_bin=None,
        transform_target=False,
        feture_elimination_num_feat=None,
        feat_agg_calculate_chosen_options_ratio=False,
        feat_agg_worked_with_high_and_low_leverage=False):
    global experiment_list
    print(name)

    x_train, x_test, y_train, y_test, data = prepare_data(exclude_columns=exclude_columns, 
                                                        features_to_use=features_to_use, 
                                                        apply_x_col=apply_x_col,
                                                        columns_to_bin=columns_to_bin,
                                                          transform_target=transform_target,
                                                         feat_agg_calculate_chosen_options_ratio=feat_agg_calculate_chosen_options_ratio,
                                                         feat_agg_worked_with_high_and_low_leverage=feat_agg_worked_with_high_and_low_leverage)

    if x_train_ is not None or y_train_ is not None:
        x_train, y_train = x_train_, y_train_

    x_train, x_test = scale_data(x_train, x_test, scaler=StandardScaler)

    results = run_models(x_train, x_test, y_train, y_test, feture_elimination_num_feat=feture_elimination_num_feat)

    best_model = results.sort_values("mean_absolute_error_test").reset_index().loc[[0], ["mean_absolute_error_train", 
                                                                                         "mape_train",
                                                                                         
                                                                                        "mean_absolute_error_test", 
                                                                                         "mape_test",
                                                                                        "index",
                                                                                        "num_feat",
                                                                                        "model_object"]].rename({0:name, "index":"model_name"})
    if experiment_list is not None:
        experiment_list = experiment_list.append(best_model)

    return results.sort_values("mean_absolute_error_test")

In [30]:
results = run(exclude_columns=None, name="Initial Run", transform_target=True)

Initial Run


In [31]:
results

,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
LassoCV,1.895949e+00,1.357928e+02,9563.731583,0.039420,2.221213,139.615025,17636.068795,0.023233,"LassoCV(alphas=None, copy_X=True, cv=None, eps...",596
RidgeCV,1.939517e+00,1.426778e+02,9420.573507,0.061600,2.303413,148.842785,17689.519655,0.009531,"RidgeCV(alphas=array([ 0.1, 1. , 10. ]), cv=N...",596
RandomForestRegressor,7.609845e-01,5.762185e+01,3762.012733,0.856604,2.439220,155.362366,17218.438146,0.015126,"(DecisionTreeRegressor(ccp_alpha=0.0, criterio...",596
DecisionTreeRegressor,3.888774e-09,3.611959e-07,0.000415,1.000000,2.362257,191.825046,17709.785203,-1.188168,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...",596


In [32]:
results = run(exclude_columns=None, name="With Agg Features",
              feat_agg_calculate_chosen_options_ratio=True,
             feat_agg_worked_with_high_and_low_leverage=True,
             transform_target=True)

With Agg Features


In [33]:
results

,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
LassoCV,1.882356e+00,1.359347e+02,9603.819632,0.037067,2.168179,139.897248,17638.152050,0.023526,"LassoCV(alphas=None, copy_X=True, cv=None, eps...",396
RidgeCV,1.854321e+00,1.400154e+02,9532.689715,0.049010,2.193565,144.822739,17699.396791,0.018092,"RidgeCV(alphas=array([ 0.1, 1. , 10. ]), cv=N...",396
RandomForestRegressor,8.243803e-01,5.656490e+01,3798.369058,0.861173,2.359000,154.598312,17438.917462,0.010628,"(DecisionTreeRegressor(ccp_alpha=0.0, criterio...",396
DecisionTreeRegressor,2.334874e-09,2.450573e-07,0.000340,1.000000,2.081811,185.457392,17882.408512,-0.795267,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...",396


In [34]:
experiment_list

,mean_absolute_error_train,mape_train,mean_absolute_error_test,mape_test,index,num_feat,model_object
Initial Run,135.792803,1.895949,139.615025,2.221213,LassoCV,596,"LassoCV(alphas=None, copy_X=True, cv=None, eps..."
With Agg Features,135.934743,1.882356,139.897248,2.168179,LassoCV,396,"LassoCV(alphas=None, copy_X=True, cv=None, eps..."


In [35]:
results = run(name="Initial Run + Feat Selection", feture_elimination_num_feat="auto", transform_target=True)
results

Initial Run + Feat Selection


,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
RFE,1.875334,138.441218,9686.606056,0.012993,2.229488,141.952350,17607.306320,0.012716,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",5
RFE,1.916159,141.040222,9438.438289,0.054084,2.223463,146.381344,17671.035135,0.014433,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",155
RFE,1.942877,142.623798,9405.617740,0.061050,2.308511,148.668668,17696.682563,0.010078,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",305
RFE,1.939517,142.677973,9420.572215,0.061600,2.303414,148.842947,17689.517761,0.009530,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",606
RFE,1.939508,142.678139,9420.551249,0.061600,2.303384,148.844485,17689.522885,0.009525,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",455


In [36]:
results = run(exclude_columns=None, name="With Agg Features + Feat Selection",
              feat_agg_calculate_chosen_options_ratio=True,
             feat_agg_worked_with_high_and_low_leverage=True,
             feture_elimination_num_feat="auto",
             transform_target=True)
results

With Agg Features + Feat Selection


,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
RFE,1.887202,138.040838,9697.543589,0.014139,2.234690,141.560716,17595.943595,0.014099,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",5
RFE,1.861798,139.068823,9545.219705,0.044532,2.118571,143.689773,17663.519595,0.020866,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",102
RFE,1.855784,139.955648,9527.895567,0.048913,2.200040,144.820486,17694.021534,0.018066,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",200
RFE,1.854321,140.015371,9532.689715,0.049010,2.193565,144.822739,17699.396791,0.018092,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",298
RFE,1.854321,140.015371,9532.689715,0.049010,2.193565,144.822739,17699.396791,0.018092,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",396


In [37]:
experiment_list

,mean_absolute_error_train,mape_train,mean_absolute_error_test,mape_test,index,num_feat,model_object
Initial Run,135.792803,1.895949,139.615025,2.221213,LassoCV,596,"LassoCV(alphas=None, copy_X=True, cv=None, eps..."
With Agg Features,135.934743,1.882356,139.897248,2.168179,LassoCV,396,"LassoCV(alphas=None, copy_X=True, cv=None, eps..."
Initial Run + Feat Selection,138.441218,1.875334,141.952350,2.229488,RFE,5,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ..."
With Agg Features + Feat Selection,138.040838,1.887202,141.560716,2.234690,RFE,5,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ..."
